In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')

FOLDERNAME = 'test_ml/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


##Подключение к устройству



In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


##Датасеты
Загрузка

In [5]:
import my_dataset

path_from = "drive/MyDrive/test_ml/dataset/train.zip"
path_to = "drive/MyDrive/test_ml/dataset/"

loaded_data = my_dataset.DataLoad(path_from, path_to)
loaded_data.data.head()

,filename,label
0,dog.4449.jpg,dog
1,dog.445.jpg,dog
2,dog.4450.jpg,dog
3,dog.4451.jpg,dog
4,dog.4452.jpg,dog


Разбиение датасета на 3 выборки: train = 80%, validation = 10%, test = 10%

In [5]:
x_train, x_valid, x_test = loaded_data.split_data()

print('Размер тренировочного набора данных', x_train.shape)
print('Размер валидационного набора данных', x_valid.shape)
print('Размер тестового набора данных', x_test.shape)

Размер тренировочного набора данных (15467, 2)
Размер валидационного набора данных (1933, 2)
Размер тестового набора данных (1934, 2)


Теперь нужно переиндексировать данные, т.к. они были разбиты и при попытке обращения к ним будет выскакивать одна и та же ошибка KeyError: 2

In [6]:
x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

Аугментация

In [7]:
from my_dataset import CustomDataset

train_dataset = CustomDataset(data_frame=x_train, img_dir=path_to + "train/", transform=my_dataset.train_augmentation)
valid_dataset = CustomDataset(data_frame=x_valid, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)
test_dataset = CustomDataset(data_frame=x_test, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)

Разметка на батчи

In [8]:
from torch.utils.data import DataLoader

batch_size = 64 # размер батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Просмотр состояния картинки из датасета

In [9]:
print(len(train_dataset))
for image, label in train_dataset:
    print(image.shape)
    print(label.shape)
    break


15467
torch.Size([3, 128, 128])
torch.Size([1])


In [10]:
# 15467 / 64 = 242 (размер датасета / размер батча)
print(len(train_loader))
for image, label in train_loader:
    print(image.shape)
    print(label.shape)
    break

242
torch.Size([64, 3, 128, 128])
torch.Size([64, 1])


##Тренировка модели
Модель представляет из себя сверточную нейросеть.

In [11]:
import model
from train import train_model

net = model.Model().to(device)

loss_history, y_pred_train, y_pred_train = train_model(train_loader, net, epochs=10, device=device)


Iteration 0, loss = 0.6959 acc = 0.5000
Iteration 10, loss = 0.6350 acc = 0.6875
Iteration 20, loss = 0.7410 acc = 0.5312
Iteration 30, loss = 0.7049 acc = 0.6094
Iteration 40, loss = 0.6715 acc = 0.6094
Iteration 50, loss = 0.6729 acc = 0.6094
Iteration 60, loss = 0.6254 acc = 0.6562
Iteration 70, loss = 0.6679 acc = 0.5938
Iteration 80, loss = 0.6533 acc = 0.6562
Iteration 90, loss = 0.5949 acc = 0.7344
Iteration 100, loss = 0.6066 acc = 0.7188
Iteration 110, loss = 0.5674 acc = 0.7344
Iteration 120, loss = 0.6562 acc = 0.6094
Iteration 130, loss = 0.6065 acc = 0.6562
Iteration 140, loss = 0.7241 acc = 0.5469
Iteration 150, loss = 0.6450 acc = 0.6094
Iteration 160, loss = 0.6057 acc = 0.7188
Iteration 170, loss = 0.5622 acc = 0.7656
Iteration 180, loss = 0.6015 acc = 0.6875
Iteration 190, loss = 0.7476 acc = 0.6250
Iteration 200, loss = 0.5925 acc = 0.7031
Iteration 210, loss = 0.6537 acc = 0.6406
Iteration 220, loss = 0.6590 acc = 0.6875
Iteration 230, loss = 0.5854 acc = 0.7188
Ite

Работа с validation набором

Работа с test набором

##Оценка качества и вывод графиков
Оценка качества будет производиться по метрике f1.

In [23]:
import sklearn.metrics as metrics

y_true_train = x_train['label']
result = metrics.f1_score(y_true_train, y_pred_train)

print("Оценка качества F1:", result)


ValueError: ignored

##Вывод графиков
Функция потерь

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20),dpi=200)

plt.title('График потерь',fontsize=15)
plt.xlabel('Итерации',fontsize=12)
plt.ylabel('Потери',fontsize=12)
plt.plot(loss_history)

plt.show()

Confusion matrix для validation набора

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

conf_matrix_valid = confusion_matrix(y_valid, y_pred_valid)
conf_matrix_labels_valid = pd.DataFrame(conf_matrix_valid, columns = ['cat', 'dog'], index = ['cat', 'dog'])
conf_matrix_labels_valid

Сonfusion matrix для test набора

In [ ]:
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
conf_matrix_labels_test = pd.DataFrame(conf_matrix_test, columns = ['cat', 'dog'], index = ['cat', 'dog'])
conf_matrix_labels_test